# Hotspot calculation demo


This notebook detects and classifies hotspots of the von Mises stress in a connected FEM mesh. Each element/node entry of the mesh receives a number of the hotspot it is member of. "0" means the element/node is not part of any hotspots. "1" means that the element/node is part of the hotspot with the highes peak, "2" the same for the second highest peak and so forth.

See [module documentation](https://pylife.readthedocs.io/en/latest/mesh/hotspot.html) further details.

In [ ]:
import numpy as np
import pylife
import pandas as pd
import scipy.stats as stats
import pylife.stress.equistress
import pylife.strength.meanstress
import pylife.mesh.meshplot
import pylife.mesh.hotspot
import matplotlib.pyplot as plt


In [ ]:
filename = 'plate_with_hole.h5'
fe_res = pd.read_hdf(filename, 'node_data')
fe_res['S13'] = np.zeros_like(fe_res['S11'])
fe_res['S23'] = np.zeros_like(fe_res['S11'])
display(fe_res)

## Equivalent stress calculation

In [ ]:
fe_res["vM"] = pylife.stress.equistress.mises(fe_res["S11"],fe_res["S22"],fe_res["S33"],fe_res["S12"],fe_res["S13"],fe_res["S23"])
display(fe_res["vM"].sort_values(ascending = False))

## Hot spot Calculation ##

In [ ]:
threshold = .5 # factor of the maximum local value
fe_res['hotspot'] = fe_res.hotspot.calc("vM", threshold)
display(fe_res[['x', 'y', 'z', 'vM', 'hotspot']])

In [ ]:
print("%d hotspots found over the threshold" % fe_res['hotspot'].max())

In [ ]:
fig, ax = plt.subplots()
fe_res.meshplot.plot(ax, 'hotspot', cmap='jet_r')
plt.show()

### First hotspot

In [ ]:
display(fe_res[fe_res['hotspot'] == 1])